In [120]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import numpy as np

In [121]:
boston = fetch_california_housing()
y = boston['target']
X = boston['data']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_data = X_train
y_data = y_train

1. 尚未构建树

In [122]:
# 初始预测值直接为0
y_pred = np.zeros(X_data.shape[0])
y_pred

array([0., 0., 0., ..., 0., 0., 0.])

In [123]:
# 计算一阶导数
garr = 2 * (y_pred - y_data)
garr

array([-2.428, -7.274, -4.112, ..., -4.442, -5.67 , -6.5  ])

In [124]:
# 计算二阶导数
harr = 2 * np.ones_like(y_data)
harr

array([2., 2., 2., ..., 2., 2., 2.])

In [125]:
# 利用X_data，计算所有切分点，候选点
splits = []
for fea in range(X_data.shape[1]):
    for val in X_data[fea]:
            splits.append((fea, val))
splits[:10]
# 每个特征的每个值都在splits里面

[(0, np.float64(0.9809)),
 (0, np.float64(19.0)),
 (0, np.float64(3.187725631768953)),
 (0, np.float64(1.1299638989169676)),
 (0, np.float64(726.0)),
 (0, np.float64(2.6209386281588447)),
 (0, np.float64(33.98)),
 (0, np.float64(-118.28)),
 (1, np.float64(4.2232)),
 (1, np.float64(33.0))]

In [126]:
# 构建树0 第0棵树，root根树
_lambda=1
_gamma = 0


# 1. 寻找最佳分裂点
bestSplit = None
maxScore = -float('inf')
subinds = None

In [127]:
def split(X_data, sp):
    # 劈裂数据集，返回左右子数据集索引
    lind = np.where(X_data[:, sp[0]] <= sp[1])[0]
    rind = list(set(range(X_data.shape[0])) - set(lind))
    return lind, rind

def calObj(garr, harr):
    return (-1.0 / 2) * sum(garr) ** 2 / (sum(harr) + _lambda) + _gamma

In [128]:
# 当前一阶导数、二阶导数的损失值
score_pre = calObj(garr, harr)
print(score_pre)

-59057.04797166389


In [129]:
for sp in splits:

    # 分割数据集，输出左右数据集的索引
    lind, rind = split(X_data, sp) 
    if len(rind) < 2 or len(lind) < 2:
        continue
    gl = garr[lind]
    gr = garr[rind]
    hl = harr[lind]
    hr = harr[rind]
    score = score_pre - calObj(gl, hl) - calObj(gr, hr)  # 切分后目标函数值下降量
    if score > maxScore:
        maxScore = score
        bestSplit = sp
        subinds = (lind, rind)

bestSplit, subinds[0][:10],subinds[1][:10]

((0, np.float64(3.187725631768953)),
 array([ 0,  3,  5,  7,  8, 10, 12, 13, 15, 17]),
 [1, 2, 4, 6, 9, 11, 14, 16, 18, 19])